In [64]:
import pandas as pd
import json
import numpy as np
import os

In [26]:
im_names = [im[:-4] for im in (os.listdir("data/Images"))]
im_names.sort()

In [38]:
'image_r_800' in im_names

True

In [15]:
data = pd.read_csv("data/FreeViewing.txt", sep='\s+')

In [16]:
data.keys()

Index(['x', 'y', 'obs', 'image', 'fixation'], dtype='object')

In [42]:
data[0:15]

,x,y,obs,image,fixation
0,216.0,186.0,1,1,1
1,199.0,194.0,1,1,2
2,212.0,189.0,1,1,3
3,252.0,155.0,1,1,4
4,344.0,199.0,1,1,5
5,256.0,70.0,1,1,6
6,344.0,165.0,1,1,7
7,262.0,86.0,1,1,8
8,334.0,159.0,1,1,9
9,330.0,137.0,1,1,10


In [35]:
list(data[0:5]['x'])

[216.0, 199.0, 212.0, 252.0, 344.0]

In [59]:
data[:15].values[:,0:2]

array([[216., 186.],
       [199., 194.],
       [212., 189.],
       [252., 155.],
       [344., 199.],
       [256.,  70.],
       [344., 165.],
       [262.,  86.],
       [334., 159.],
       [330., 137.],
       [335., 171.],
       [336., 191.],
       [240.,  68.],
       [ nan,  nan],
       [ nan,  nan]])

In [89]:
np.isnan(np.float64(np.nan))

True

In [90]:
def clean_fix(fixs, min_dur=20):
    fixation = []
    try:
        for x,y in fixs:
            if x == None or np.isnan(x) or y == None or np.isnan(y):
                continue
            x = float(x)
            y = float(y)
            if len(fixation) == 0:
                fixation.append([y,x])
                continue
            fy, fx = fixation[-1]
      
            if abs(fy - float(y)) < 20 and abs(fx - float(x)) < 20:
                fy = round((fy + y)/2, 2)
                fx = round((fx + x)/2, 2)
                fixation[-1] = [fy, fx]
            else:
                fixation.append([y,x])
      
    except:
        print('only single fixation found')

    return fixation

In [95]:
anno_dict = {f'image_r_{i}': {'fixations': []} for i in range(1,801)}

for i in range(0, len(data), 15):
    im_name = f'image_r_{int(data[i:i+1].image)}'
    fixation = clean_fix(data[i:i+15].values[:,0:2])
    if len(fixation) > 1:
        anno_dict[im_name]['fixations'].append(fixation)

In [96]:
k = list(anno_dict.keys())

In [97]:
len(k)

800

In [104]:
fixlens = []
num_persons = []
for _, d in anno_dict.items():
    num_persons.append(len(d['fixations']))
    for fixs in d['fixations']:
        fixlens.append(len(fixs))

In [100]:
min(fixlens), max(fixlens)

(2, 14)

In [101]:
sum(fixlens) / len(fixlens)

5.228952150211993

In [106]:
from statistics import mode
mode(fixlens)

6

In [105]:
min(num_persons), max(num_persons)

(17, 22)

In [102]:
with open('VIU_annotations.json', 'w') as fp:
    json.dump(anno_dict, fp)